In [1]:
import numpy as np
from scipy.stats import entropy
from scipy import ndimage
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [2]:
l1_dist = lambda x1,x2: np.sum(np.abs(x1 - x2), axis=tuple(range(len(x1.shape))[1:]))
l2_dist = lambda x1,x2: np.sum((x1-x2)**2, axis=tuple(range(len(x1.shape))[1:]))**.5

In [3]:
# Note: KL-divergence is not symentric.
# Designed for probability distribution (e.g. softmax output).
def kl(x1, x2):
    assert x1.shape == x2.shape
    # x1_2d, x2_2d = reshape_2d(x1), reshape_2d(x2)

    # Transpose to [?, #num_examples]
    x1_2d_t = x1.transpose()
    x2_2d_t = x2.transpose()

    # pdb.set_trace()
    e = entropy(x1_2d_t, x2_2d_t)
    e[np.where(e==np.inf)] = 2
    return e

In [4]:
def reduce_precision_py(x, npp):
    """
    Reduce the precision of image, the numpy version.
    :param x: a float tensor, which has been scaled to [0, 1].
    :param npp: number of possible values per pixel. E.g. it's 256 for 8-bit gray-scale image, and 2 for binarized image.
    :return: a tensor representing image(s) with lower precision.
    """
    # Note: 0 is a possible value too.
    npp_int = npp - 1
    x_int = np.rint(x * npp_int)
    x_float = x_int / npp_int
    return x_float

In [5]:
def median_filter_py(x, width, height=-1):
    """
    Median smoothing by Scipy.
    :param x: a tensor of image(s)
    :param width: the width of the sliding window (number of pixels)
    :param height: the height of the window. The same as width by default.
    :return: a modified tensor with the same shape as x.
    """
    if height == -1:
        height = width
    var = ndimage.filters.median_filter(x, size=(1,width,height,1), mode='reflect')
    #print("inside median filter")
    #print(type(var))
    return torch.from_numpy(var)

In [6]:
# Squeezers implemented in OpenCV
# OpenCV expects uint8 as image data type.
def opencv_wrapper(imgs, opencv_func, argv):
    ret_imgs = []
    imgs_copy = imgs

    if imgs.shape[3] == 1:
        imgs_copy = np.squeeze(imgs)

    for img in imgs_copy:
        img_uint8 = np.clip(np.rint(img * 255), 0, 255).astype(np.uint8)
        ret_img = opencv_func(*[img_uint8]+argv)
        if type(ret_img) == tuple:
            ret_img = ret_img[1]
        ret_img = ret_img.astype(np.float32) / 255.
        ret_imgs.append(ret_img)
    ret_imgs = np.stack(ret_imgs)

    if imgs.shape[3] == 1:
        ret_imgs = np.expand_dims(ret_imgs, axis=3)

    return 

In [7]:
def bit_depth_py(x, bits):
    precisions = 2**bits
    return reduce_precision_py(x, precisions)

In [8]:
def non_local_means_color_py(imgs, search_window, block_size, photo_render):
    import cv2
    ret_imgs = opencv_wrapper(imgs, cv2.fastNlMeansDenoisingColored, [None,photo_render,photo_render,block_size,search_window])
    return ret_imgs

In [9]:
m = nn.Softmax(dim=1)

In [10]:
def get_distance(model, dataset, X1):
    #X1_pred = model.predict(X1)
    X1_pred = m(model(X1))
    vals_squeezed = []

    if dataset == 'mnist':
        X1_seqeezed_bit = bit_depth_py(X1.cpu(), 1)
        #print(type(X1_seqeezed_bit))
        
        #model.predict is tf based. need torch based softmax. 
        #vals_squeezed.append(model.predict(X1_seqeezed_bit))
        vals_squeezed.append(m(model((X1_seqeezed_bit).to(device))))
        X1_seqeezed_filter_median = median_filter_py(X1.cpu(), 2)
        #print(("outside func",type(X1_seqeezed_filter_median)))
        vals_squeezed.append(m(model(X1_seqeezed_filter_median.to(device))))

    else:
        X1_seqeezed_bit = bit_depth_py(X1.cpu(), 5)
        #print(type(X1_seqeezed_bit))
        vals_squeezed.append(m(model((X1_seqeezed_bit).to(device))))
        X1_seqeezed_filter_median = median_filter_py(X1.cpu(), 2)
        #vals_squeezed.append(model(torch.from_numpy(X1_seqeezed_filter_median).float().to(device)))
        vals_squeezed.append(m(model((X1_seqeezed_filter_median).to(device))))
        #X1_seqeezed_filter_local = non_local_means_color_py(X1.cpu().numpy(), 13, 3, 2)
        #vals_squeezed.append(m(model((X1_seqeezed_filter_local).to(device))))

    dist_array = []
    for val_squeezed in vals_squeezed:
        dist = np.sum(np.abs(X1_pred.cpu().detach().numpy() - val_squeezed.cpu().detach().numpy()), axis=tuple(range(len(X1_pred.shape))[1:]))
        dist_array.append(dist)

    dist_array = np.array(dist_array)
    return np.max(dist_array, axis=0)

In [11]:
def get_distance_test(model, dataset, X1):
    #X1_pred = model.predict(X1)
    X1_pred = m(model(X1))
    vals_squeezed = []

    if dataset == 'mnist':
        X1_seqeezed_bit = bit_depth_py(X1.detach().cpu(), 1)
        #print(type(X1_seqeezed_bit))
        
        #model.predict is tf based. need torch based softmax. 
        #vals_squeezed.append(model.predict(X1_seqeezed_bit))
        vals_squeezed.append(m(model((X1_seqeezed_bit).to(device))))
        X1_seqeezed_filter_median = median_filter_py(X1.detach().cpu(), 2)
        #print(("outside func",type(X1_seqeezed_filter_median)))
        vals_squeezed.append(m(model(X1_seqeezed_filter_median.to(device))))

    else:
        X1_seqeezed_bit = bit_depth_py(X1.detach().cpu(), 5)
        #print(type(X1_seqeezed_bit))
        vals_squeezed.append(m(model((X1_seqeezed_bit).to(device))))
        X1_seqeezed_filter_median = median_filter_py(X1.detach().cpu(), 2)
        #vals_squeezed.append(model(torch.from_numpy(X1_seqeezed_filter_median).float().to(device)))
        vals_squeezed.append(m(model((X1_seqeezed_filter_median).to(device))))
        #X1_seqeezed_filter_local = non_local_means_color_py(X1.cpu().numpy(), 13, 3, 2)
        #vals_squeezed.append(m(model((X1_seqeezed_filter_local).to(device))))

    dist_array = []
    for val_squeezed in vals_squeezed:
        dist = np.sum(np.abs(X1_pred.cpu().detach().numpy() - val_squeezed.cpu().detach().numpy()), axis=tuple(range(len(X1_pred.shape))[1:]))
        dist_array.append(dist)

    dist_array = np.array(dist_array)
    return np.max(dist_array, axis=0)

In [12]:
def compute_distance_test(model, dataset, X):
    distances = get_distance_test(model, dataset, X)
    return distances

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib
import matplotlib.pyplot as plt
from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.torch.attacks.projected_gradient_descent import (
    projected_gradient_descent,
)
import gc
from captum.attr import *
import quantus
from torch.utils.data import DataLoader
import gc
import torchvision.transforms as transforms
from quantus.functions.perturb_func import baseline_replacement_by_indices
from art.attacks.evasion import CarliniLInfMethod
import torch.optim as optim
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import SaliencyMapMethod
import warnings
warnings.filterwarnings('ignore')

In [14]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
from typing import Any, Dict, Optional, Sequence, Tuple, Union, List

def infer_attribution_axes(a_batch: np.ndarray, x_batch: np.ndarray)-> Sequence[int]:
    """
    Reference: quantus library
    Infers the axes in x_batch that are covered by a_batch.
    Parameters
    ----------
    x_batch: np.ndarray
        A np.ndarray which contains the input data that are explained.
    a_batch: np.ndarray
        An array which contains pre-computed attributions i.e., explanations.
    Returns
    -------
    np.ndarray
        The axes inferred.
    """
    # TODO: Adapt for batched processing.

    if a_batch.shape[0] != x_batch.shape[0]:
        raise ValueError(
            f"a_batch and x_batch must have same number of batches ({a_batch.shape[0]} != {x_batch.shape[0]})"
        )

    if a_batch.ndim > x_batch.ndim:
        raise ValueError(
            "Attributions need to have <= dimensions than inputs, but {} > {}".format(
                a_batch.ndim, x_batch.ndim
            )
        )

    # TODO: We currently assume here that the batch axis is not carried into the perturbation functions.
    a_shape = [s for s in np.shape(a_batch)[1:] if s != 1]
    x_shape = [s for s in np.shape(x_batch)[1:]]

    if a_shape == x_shape:
        return np.arange(0, len(x_shape))

    # One attribution value per sample
    if len(a_shape) == 0:
        return np.array([])

    x_subshapes = [
        [x_shape[i] for i in range(start, start + len(a_shape))]
        for start in range(0, len(x_shape) - len(a_shape) + 1)
    ]
    if x_subshapes.count(a_shape) < 1:

        # Check that attribution dimensions are (consecutive) subdimensions of inputs
        raise ValueError(
            "Attribution dimensions are not (consecutive) subdimensions of inputs:  "
            "inputs were of shape {} and attributions of shape {}".format(
                x_batch.shape, a_batch.shape
            )
        )
    elif x_subshapes.count(a_shape) > 1:

        # Check that attribution dimensions are (unique) subdimensions of inputs.
        # Consider potentially expanded dims in attributions.

        if a_batch.ndim == x_batch.ndim and len(a_shape) < a_batch.ndim:
            a_subshapes = [
                [np.shape(a_batch)[1:][i] for i in range(start, start + len(a_shape))]
                for start in range(0, len(np.shape(a_batch)[1:]) - len(a_shape) + 1)
            ]
            if a_subshapes.count(a_shape) == 1:

                # Inferring channel shape.
                for dim in range(len(np.shape(a_batch)[1:]) + 1):
                    if a_shape == np.shape(a_batch)[1:][dim:]:
                        return np.arange(dim, len(np.shape(a_batch)[1:]))
                    if a_shape == np.shape(a_batch)[1:][:dim]:
                        return np.arange(0, dim)

            raise ValueError(
                "Attribution axes could not be inferred for inputs of "
                "shape {} and attributions of shape {}".format(
                    x_batch.shape, a_batch.shape
                )
            )

        raise ValueError(
            "Attribution dimensions are not unique subdimensions of inputs:  "
            "inputs were of shape {} and attributions of shape {}."
            "Please expand attribution dimensions for a unique solution".format(
                x_batch.shape, a_batch.shape
            )
        )
    else:
        # Infer attribution axes.
        for dim in range(len(x_shape) + 1):
            if a_shape == x_shape[dim:]:
                return np.arange(dim, len(x_shape))
            if a_shape == x_shape[:dim]:
                return np.arange(0, dim)

    raise ValueError(
        "Attribution axes could not be inferred for inputs of "
        "shape {} and attributions of shape {}".format(x_batch.shape, a_batch.shape)
    )


In [16]:
def modify_x_based_on_attribution(x_batch, a_batch_benign, d_type, top_k = 100):
    #get attribution axes
    a_axes = infer_attribution_axes(a_batch_benign, x_batch)
    perturbed_arr = []

    #modify each sample one-by-one
    for x,a in zip(x_batch, a_batch_benign):
        #flatten the attribution 
        a = a.flatten()
        #get indices of sorted attributions (ascending)
        a_ind = np.argsort(a)
        #get indices of top_k 
        a_ix = a_ind[-top_k:]
        if d_type=="adv":
            x = x.cpu()
            x_perturbed = baseline_replacement_by_indices(arr = x.detach().numpy(), indices = a_ix, indexed_axes = a_axes, perturb_baseline = "black")
            if (x.detach().numpy().flatten() != x_perturbed.flatten()).any():
                perturbed_arr.append(torch.from_numpy(x_perturbed))
    
        else:
            x_perturbed = baseline_replacement_by_indices(arr = x.cpu().numpy(), indices = a_ix, indexed_axes = a_axes, perturb_baseline = "black")
            if (x.cpu().numpy().flatten() != x_perturbed.flatten()).any():
                perturbed_arr.append(torch.from_numpy(x_perturbed))
    
    new_batch = torch.stack(perturbed_arr)
    return new_batch.to('cuda')

In [17]:
def make_noise(x_batch, y_batch, spread):
    new_x_batch = []
    for x in x_batch:
        x = x.data.cpu().numpy()
        stdev = spread * (np.max(x)-np.min(x))
        noise = np.random.normal(0, stdev, x.shape).astype(np.float32)
        x_plus_noise = x + noise
        x_plus_noise = np.clip(x_plus_noise, 0, 1)
        x_plus_noise = torch.from_numpy(x_plus_noise).cpu()
        new_x_batch.append(x_plus_noise)
    new_batch = torch.stack(new_x_batch).to(device)
    return new_batch

In [18]:
# Define uniform noise function
def add_uniform_noise(image):
    # Generate uniform noise with mean 0 and standard deviation 25
    noise = np.random.uniform(low=-0.5, high=0.5, size=image.shape).astype(np.float32)
    noisy_image = np.clip(image + noise, 0, 1).astype(np.uint8)
    return noisy_image

def uniform_noise(x_batch, y_batch): 
    # Convert batch of images to numpy array
    images = x_batch.detach().cpu().numpy().transpose(0, 2, 3, 1) * 1.0
    # Add Poisson noise to each image in the batch
    noisy_images = [add_uniform_noise(image) for image in images]
    # Convert noisy images back to Tensor format
    noisy_inputs = torch.from_numpy(np.array(noisy_images).transpose(0, 3, 1, 2) / 1.0).float()
    return noisy_inputs.to(device)

In [19]:
def compute_mean_abs_dev(attr): 
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        avg = np.mean(a)
        deviation = a - avg 
        absolute_deviation = np.abs(deviation)
        result = np.mean(absolute_deviation)
        scores.append(result)
    return scores    

In [20]:
def compute_median_abs_dev(attr): 
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        med = np.median(a)
        deviation = a - med 
        abs_deviation = np.abs(deviation)
        result = np.median(abs_deviation)
        scores.append(result)
    return scores 

In [21]:
def compute_iqr(attr):
    #inter-quartile range
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        score_75 = np.percentile(a, 75)
        score_25 = np.percentile(a, 25)
        score_qt = score_75 - score_25
        scores.append(score_qt)
    return scores
    

In [22]:
#relative measure of dispersion
def compute_coef_var(attr):
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        m = np.mean(a)
        st = np.std(attr[i])
        sc = m/st
        scores.append(sc)
    return scores

In [23]:
#relative measure of dispersion

## Coefficient of quartile dev

def compute_coef_iqr(attr):
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        score_75 = np.percentile(a, 75)
        score_25 = np.percentile(a, 25)
        score_qt = (score_75 - score_25)/(score_75 + score_25)
        scores.append(score_qt)
    return scores
    

In [24]:
def make_attack(x_batch, y_batch, eps, normal_model): 
    
    alpha = eps/10
    steps = int(alpha*eps)
    images_pgd = projected_gradient_descent(normal_model, x_batch, eps, alpha, steps, np.inf)
    _, y_pred_pgd = normal_model(images_pgd).max(1)
    index = (y_pred_pgd != y_batch)
    pgd_images = images_pgd[index]
    y_pred_pgd = y_pred_pgd[index]
    return pgd_images, y_pred_pgd

In [25]:
def make_fgsm_attack(x_batch, y_batch, eps, normal_model): 
    
    images_pgd = fast_gradient_method(normal_model, x_batch, eps, np.inf)
    _, y_pred_pgd = normal_model(images_pgd).max(1)
    index = (y_pred_pgd != y_batch)
    pgd_images = images_pgd[index]
    y_pred_pgd = y_pred_pgd[index]
    return pgd_images, y_pred_pgd

In [26]:
def make_cw_attack(attack, x_batch, y_batch, normal_model):
    #convert tensor to numpy
    #x_batch=x_batch.to('cuda')
    #_, y_pred_pgd = normal_model(x_batch).max(1)
    x_batch = x_batch.cpu().numpy()
    x_test_adv = attack.generate(x=x_batch)
    #convert the nd array back to tensor
    x_test = torch.from_numpy(x_test_adv).to('cuda')
    _, y_test = normal_model(x_test).max(1)
    index = (y_test != y_batch)
    adv_images = x_test[index]
    y_pred_adv = y_test[index]
    return adv_images, y_pred_adv

In [27]:
def make_bim_attack(attack, x_batch, y_batch, normal_model):
    #convert tensor to numpy
    #x_batch=x_batch.to('cuda')
    #_, y_pred_pgd = normal_model(x_batch).max(1)
    x_batch = x_batch.cpu().numpy()
    x_test_adv = attack.generate(x=x_batch)
    #convert the nd array back to tensor
    x_test = torch.from_numpy(x_test_adv).to('cuda')
    _, y_test = normal_model(x_test).max(1)
    index = (y_test != y_batch)
    adv_images = x_test[index]
    y_pred_adv = y_test[index]
    return adv_images, y_pred_adv

In [28]:
#for natural and adversarial LeNet Model 
class LeNet_normal(torch.nn.Module):
    """Network architecture from: https://github.com/ChawDoe/LeNet5-MNIST-PyTorch."""
    def __init__(self):
        super().__init__()
        self.conv_1 = torch.nn.Conv2d(1, 6, 5)
        self.pool_1 = torch.nn.MaxPool2d(2, 2)
        self.relu_1 = torch.nn.ReLU()
        self.conv_2 = torch.nn.Conv2d(6, 16, 5)
        self.pool_2 = torch.nn.MaxPool2d(2, 2)
        self.relu_2 = torch.nn.ReLU()
        self.fc_1 = torch.nn.Linear(256, 120)
        self.relu_3 = torch.nn.ReLU()
        self.fc_2 = torch.nn.Linear(120, 84)
        self.relu_4 = torch.nn.ReLU()
        self.fc_3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool_1(self.relu_1(self.conv_1(x)))
        x = self.pool_2(self.relu_2(self.conv_2(x)))
        x = x.view(x.shape[0], -1)
        x = self.relu_3(self.fc_1(x))
        x = self.relu_4(self.fc_2(x))
        x = self.fc_3(x)
        return x

In [29]:
#from rev2.cifar10.model_utils import resnet50, CIFAR10_RESNET50_CKPT_PATH

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, out_keys=None):
        out = {}
        x = self.conv1(x)
        out["c1"] = x
        x = self.bn1(x)
        out["bn1"] = x
        x = F.relu(x)
        out["r1"] = x

        x = self.layer1(x)
        out["l1"] = x
        x = self.layer2(x)
        out["l2"] = x
        x = self.layer3(x)
        out["l3"] = x
        x = self.layer4(x)
        out["l4"] = x

        x = F.avg_pool2d(x, 4)
        out["gvp"] = x
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        out["fc"] = x

        if out_keys is None:
            return x
        res = {}
        for key in out_keys:
            res[key] = out[key]
        return res


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])


def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])


def resnet50():
    return ResNet(Bottleneck, [3,4,6,3])


def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])


def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])


def test():
    net = ResNet18()
    y = net(torch.randn(1,3,32,32))
    print(y.size())

In [30]:
#mnist
def compute_benign_mnist(train_loader_mnist, normal_model): 
    
    print("Computing metrics for benign")
    #statistics benign
    medianAbs_ben = []
    meanAbs_ben = []
    iqr_ben = []
    coef_var_ben=[]
    coef_iqr_ben = []

    #input squeeze benign
    logit_ben1 = []
    logit_ben2 = []
    logit_ben3 = []

    #attribution robustness
    benign_attribution_gaussian1 = []
    benign_attribution_gaussian2 = []
    benign_attribution_gaussian3 = []
    benign_attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    fs = []

    for step, (x_batch, y_batch) in enumerate(train_loader_mnist):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: feature squeezing 
        d = compute_distance_test(normal_model, "mnist", x_batch)
        fs.extend(d)
        
        #approach: statistics
        x_logits = normal_model(x_batch)
        a_batch_benign = (Saliency(normal_model).attribute(inputs=x_batch, target=y_batch).sum(axis=1).cpu().numpy())
        meanAbs_ben += compute_mean_abs_dev(a_batch_benign)
        medianAbs_ben += compute_median_abs_dev(a_batch_benign)
        iqr_ben += compute_iqr(a_batch_benign)
        coef_var_ben += compute_coef_var(a_batch_benign)
        coef_iqr_ben += compute_coef_iqr(a_batch_benign)
        
        #approach: input squeeze
        a_batch_benign = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=x_batch, target=y_batch).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(x_batch, a_batch_benign, top_k = 100, d_type="not-adv")
        x_modified2 = modify_x_based_on_attribution(x_batch, a_batch_benign, top_k = 150, d_type="not-adv")
        x_modified3 = modify_x_based_on_attribution(x_batch, a_batch_benign, top_k = 180, d_type="not-adv")
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit_ben1.extend(diff1.detach().cpu().numpy())
        logit_ben2.extend(diff2.detach().cpu().numpy())
        logit_ben3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch_benign = quantus.explain(
            model=normal_model, inputs=x_batch, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(x_batch, y_batch, spread = 0.005)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(x_batch, y_batch, spread = 0.01)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(x_batch, y_batch, spread = 0.05)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(x_batch, y_batch)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch_benign, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch_benign, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch_benign, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch_benign, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_uniform.append(c)
        
        #save the results in csv
        if step > 50: #500
            
            df = pd.DataFrame([
            medianAbs_ben,
            meanAbs_ben, 
            iqr_ben, 
            coef_var_ben, 
            coef_iqr_ben, 
            logit_ben1,
            logit_ben2,
            logit_ben3,
            benign_attribution_gaussian1,
            benign_attribution_gaussian2,
            benign_attribution_gaussian3,
            benign_attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform, 
            fs], index = [
            "Benign Median Absolute Dev", 
            "Benign Mean Absolute Dev", 
            "Benign IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze top 100",
            "Input squeeze top 150",
            "Input squeeze top 180",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
                "FS"
        ])
            
            df.to_csv("Benign_MNIST_Metrics.csv")
            break

In [31]:
#cifar

def compute_benign_cifar(train_loader_cifar, normal_model): 
    
    print("Computing metrics for benign")
    #statistics benign
    medianAbs_ben = []
    meanAbs_ben = []
    iqr_ben = []
    coef_var_ben=[]
    coef_iqr_ben = []

    #input squeeze benign
    logit_ben1 = []
    logit_ben2 = []
    logit_ben3 = []

    #attribution robustness
    benign_attribution_gaussian1 = []
    benign_attribution_gaussian2 = []
    benign_attribution_gaussian3 = []
    benign_attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    
    fs = []

    for step, (x_batch, y_batch) in enumerate(train_loader_cifar):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        
        #approach: feature squeezing 
        d = compute_distance_test(normal_model, "cifar", x_batch)
        fs.extend(d)
        
        #approach: statistics
        x_logits = normal_model(x_batch)
        a_batch_benign = (Saliency(normal_model).attribute(inputs=x_batch, target=y_batch).sum(axis=1).cpu().numpy())
        meanAbs_ben += compute_mean_abs_dev(a_batch_benign)
        medianAbs_ben += compute_median_abs_dev(a_batch_benign)
        iqr_ben += compute_iqr(a_batch_benign)
        coef_var_ben += compute_coef_var(a_batch_benign)
        coef_iqr_ben += compute_coef_iqr(a_batch_benign)
        
        #approach: input squeeze
        a_batch_benign = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=x_batch, target=y_batch).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(x_batch, a_batch_benign, top_k = 400, d_type="not-adv")
        x_modified2 = modify_x_based_on_attribution(x_batch, a_batch_benign, top_k = 450, d_type="not-adv")
        x_modified3 = modify_x_based_on_attribution(x_batch, a_batch_benign, top_k = 500, d_type="not-adv")
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit_ben1.extend(diff1.detach().cpu().numpy())
        logit_ben2.extend(diff2.detach().cpu().numpy())
        logit_ben3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch_benign = quantus.explain(
            model=normal_model, inputs=x_batch, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(x_batch, y_batch, spread = 0.05)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(x_batch, y_batch, spread = 0.10)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(x_batch, y_batch, spread = 0.15)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(x_batch, y_batch)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch_benign, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch_benign, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch_benign, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch_benign, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_uniform.append(c)
        
        #save the results in csv
        if step > 50: #500
            
            df = pd.DataFrame([
            medianAbs_ben,
            meanAbs_ben, 
            iqr_ben, 
            coef_var_ben, 
            coef_iqr_ben, 
            logit_ben1,
            logit_ben2,
            logit_ben3,
            benign_attribution_gaussian1,
            benign_attribution_gaussian2,
            benign_attribution_gaussian3,
            benign_attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform, 
            fs], index = [
            "Benign Median Absolute Dev", 
            "Benign Mean Absolute Dev", 
            "Benign IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze top 400",
            "Input squeeze top 450",
            "Input squeeze top 500",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
                "Feature squeeze"
        ])
            
            df.to_csv("Benign_CIFAR_Metrics.csv")
            break

In [32]:
#imagenet

def compute_benign(train_loader, normal_model): 
    #compute metrics for svhn
    
    print("Computing metrics for benign")
    #statistics benign
    medianAbs_ben = []
    meanAbs_ben = []
    iqr_ben = []
    coef_var_ben=[]
    coef_iqr_ben = []

    #input squeeze benign
    logit_ben1 = []
    logit_ben2 = []
    logit_ben3 = []

    #attribution robustness
    benign_attribution_gaussian1 = []
    benign_attribution_gaussian2 = []
    benign_attribution_gaussian3 = []
    benign_attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    
    fs=[]

    for step, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        
        d = compute_distance_test(normal_model, "imagenet", x_batch)
        fs.extend(d)
        
        #approach: statistics
        x_logits = normal_model(x_batch)
        a_batch_benign = (Saliency(normal_model).attribute(inputs=x_batch, target=y_batch).sum(axis=1).cpu().numpy())
        meanAbs_ben += compute_mean_abs_dev(a_batch_benign)
        medianAbs_ben += compute_median_abs_dev(a_batch_benign)
        iqr_ben += compute_iqr(a_batch_benign)
        coef_var_ben += compute_coef_var(a_batch_benign)
        coef_iqr_ben += compute_coef_iqr(a_batch_benign)
        
        #approach: input squeeze
        a_batch_benign = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=x_batch, target=y_batch).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(x_batch, a_batch_benign, top_k = 5000, d_type="not-adv")
        x_modified2 = modify_x_based_on_attribution(x_batch, a_batch_benign, top_k = 10000, d_type="not-adv")
        x_modified3 = modify_x_based_on_attribution(x_batch, a_batch_benign, top_k = 30000, d_type="not-adv")
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit_ben1.extend(diff1.detach().cpu().numpy())
        logit_ben2.extend(diff2.detach().cpu().numpy())
        logit_ben3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch_benign = quantus.explain(
            model=normal_model, inputs=x_batch, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(x_batch, y_batch, spread = 0.15)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(x_batch, y_batch, spread = 0.25)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(x_batch, y_batch, spread = 0.35)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(x_batch, y_batch)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch_benign, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch_benign, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch_benign, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch_benign, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_uniform.append(c)
        
        #save the results in csv
        if step > 100: #500
            break
    
    df = pd.DataFrame([
            medianAbs_ben,
            meanAbs_ben, 
            iqr_ben, 
            coef_var_ben, 
            coef_iqr_ben, 
            logit_ben1,
            logit_ben2,
            logit_ben3,
            benign_attribution_gaussian1,
            benign_attribution_gaussian2,
            benign_attribution_gaussian3,
            benign_attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform, 
    fs], index = [
            "Benign Median Absolute Dev", 
            "Benign Mean Absolute Dev", 
            "Benign IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze top 100",
            "Input squeeze top 150",
            "Input squeeze top 200",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
        "feature squeeze"
        ])
    
    df.to_csv("Benign_IMAGENET_Metrics.csv")

In [33]:
def load_imagenet_model():
    model=torchvision.models.mobilenet_v3_small(weights=True).to(device)
    model.to('cuda')
    model.train(False)
    return model

In [34]:
def main_imagenet(): 
    
    images = '/home/db1702/Downloads/imagenet-mini/val/'
    normal_model = load_imagenet_model()
    normal_model.to(device)
    normal_model.eval()
    
    # the validation transforms
    valid_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.5, 0.5, 0.5],
            std=[0.5, 0.5, 0.5]
        )])

    #get dataset
    train = torchvision.datasets.ImageFolder(images, transform=valid_transform)
    train_loader = DataLoader(train, shuffle=True, batch_size = 5)

    
    compute_benign(train_loader, normal_model) 

In [35]:
def load_mnist_model(path):
    model = LeNet_normal()
    model.to(device)
    model.load_state_dict(torch.load(path))
    model.to('cuda')
    model.train(False)
    return model

In [36]:
def main_mnist():
    
    
    #get model
    path = "mnist_model.pth"
    normal_model = load_mnist_model(path)
    normal_model.to(device)
    normal_model.eval()
    
    #get dataset
    test_set = torchvision.datasets.MNIST(root='./sample_data', train=False, transform=torchvision.transforms.ToTensor(), download=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=10, pin_memory=True)
    
    
    compute_benign_mnist(test_loader, normal_model) 

In [37]:
def load_cifar_model(path):
    model = resnet50()
    ckpt_dict = torch.load(path, lambda storage, loc: storage)
    model.load_state_dict(ckpt_dict)
    model.to('cuda')
    model.train(False)
    return model

In [38]:
def main_cifar(): 
    
    #load cifar model and set to eval
    path = "cifar.ckpt"
    normal_model = load_cifar_model(path)
    normal_model.to(device)
    normal_model.eval()
    
    #get dataset
    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=torchvision.transforms.ToTensor())
    test_loader = DataLoader(testset, shuffle=True, batch_size=10)
    
    compute_benign_cifar(test_loader, normal_model) 

In [ ]:
main_cifar()

In [ ]:
main_mnist()

In [ ]:
main_imagenet()

# compute fpr for all dataset

In [111]:
df_cifar = pd.read_csv("Benign_CIFAR_Metrics.csv")
df_mnist = pd.read_csv("Benign_MNIST_Metrics.csv")
df_imagenet = pd.read_csv("Benign_IMAGENET_Metrics.csv")

# CIFAR

In [112]:
feature_squeezer = df_cifar.iloc[16].values.flatten().tolist()[1:]
threshold = [1.15526, 1.14371, 0.3224780]

In [113]:
def compute_metrics(ben, threshold): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in ben: 
        if value<threshold:
            TN += 1
        else: 
            FP += 1
    
    
    return (FP/(TN+FP))*100

In [114]:
for th in threshold:
    FPR = compute_metrics(feature_squeezer, th)
    print(FPR)

11.923076923076923
11.923076923076923
18.076923076923077


In [115]:
#using all detectors

squeezer_input = df_cifar.iloc[7].values.flatten().tolist()[1:]
logit_gaussian3 = df_cifar.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_cifar.iloc[10].values.flatten().tolist()[1:]
medianAbsDev = df_cifar.iloc[0].values.flatten().tolist()[1:]
meanAbsDev = df_cifar.iloc[1].values.flatten().tolist()[1:]
iqr = df_cifar.iloc[2].values.flatten().tolist()[1:]
coef_iqr = df_cifar.iloc[4].values.flatten().tolist()[1:]
coef_var = df_cifar.iloc[3].values.flatten().tolist()[1:]

def compute_metrics1(meanAb, a, b, medianAb, c,d, coefiqr, e, f, coefvar, g, h, ap1, i, j, ap2a, k, l, ap2b, m, n, iqr, o, p): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1, value2, value3, value4, value5, value6, value7, value8 in zip(meanAb,medianAb,coefiqr,coefvar,ap1,ap2a,ap2b, iqr): 
        if value1<a or value1>b:
            FP += 1
        else:
            if value2<c or value2>d:
                FP +=1
            else: 
                if value3<e or value3>f:
                    FP +=1
                else:
                    if value4<g or value4>h:
                        FP +=1
                    else: 
                        if value5<i or value5>j:
                            FP +=1
                        else: 
                            if value6<k or value6>l:
                                FP +=1
                            else:
                                if value7<m or value7>n:
                                    FP +=1
                                else:
                                    if value8<o or value8>p:
                                        FP +=1

    
    
    return (FP/(len(meanAb)))*100

In [116]:
a=[1.3,1.85,2]
b=[7,7,6]
c=[0.9,1.08,1.23]
d=[5,4,4]
e=[0.461,0.491,0.502]
f=[0.8,0.8,10]
g=[0.42,0.42,0.42]
h=[1.27,1.19,1.146]

i=[48,68,78]
j=[230,230,230]

#logitgaussian3
k=[44,64,76]
l=[405,405,405]

#attrgaussian3
m=[2200,2800,3100]
n=[8800,8800,8800]


#iqr
o=[2,2.5,2.8]
p=[10,10,10]


print(compute_metrics1(meanAbsDev, a[0], b[0],
                      medianAbsDev ,c[0], d[0],
                      coef_iqr , e[0], f[0], 
                      coef_var, g[0], h[0],
                      squeezer_input,i[0],j[0], 
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0], 
                      iqr, o[0], p[0]))

print(compute_metrics1(meanAbsDev, a[1], b[1], 
                      medianAbsDev, c[1], d[1], 
                      coef_iqr, e[1], f[1],
                      coef_var, g[1], h[1], 
                      squeezer_input,i[1],j[1], 
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1], 
                      iqr, o[1], p[1]))

print(compute_metrics1(meanAbsDev, a[2], b[2],
                      medianAbsDev , c[2], d[2],
                      coef_iqr , e[2], f[2], 
                      coef_var,g[2], h[2],
                      squeezer_input,i[2],j[2], 
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2], 
                      iqr, o[2], p[2]))


6.923076923076923
23.46153846153846
39.23076923076923


In [117]:
#using only approach 1

#using only approach 1 and 2 detectors

squeezer_input = df_cifar.iloc[7].values.flatten().tolist()[1:]

def compute_metrics2(ap1, i, j): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value5 in (ap1):
        if value5<i or value5>j:
            FP +=1
        

    return (FP/(len(ap1)))*100


i=[48,68,78]
j=[230,230,230]

#logitgaussian3
k=[44,64,76]
l=[405,405,405]

#attrgaussian3
m=[2200,2800,3100]
n=[8800,8800,8800]



print(compute_metrics2(squeezer_input,i[0],j[0]))

print(compute_metrics2(squeezer_input,i[1],j[1]))

print(compute_metrics2(squeezer_input,i[2],j[2]))


3.076923076923077
6.730769230769231
11.346153846153847


In [118]:
#using only approach 1 and 2 detectors

squeezer_input = df_cifar.iloc[7].values.flatten().tolist()[1:]
logit_gaussian3 = df_cifar.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_cifar.iloc[10].values.flatten().tolist()[1:]

def compute_metrics2(ap1, i, j, ap2a, k, l, ap2b, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value5, value6, value7 in zip(ap1,ap2a,ap2b):
        if value5<i or value5>j:
            FP +=1
        else: 
            if value6<k or value6>l:
                FP +=1
            else:
                if value7<m or value7>n:
                    FP +=1

    return (FP/(len(ap1)))*100


i=[48,68,78]
j=[230,230,230]

#logitgaussian3
k=[44,64,76]
l=[405,405,405]

#attrgaussian3
m=[2200,2800,3100]
n=[8800,8800,8800]



print(compute_metrics2(squeezer_input,i[0],j[0], 
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0]))

print(compute_metrics2(squeezer_input,i[1],j[1], 
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1]))

print(compute_metrics2(squeezer_input,i[2],j[2], 
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2]))


5.0
14.615384615384617
27.115384615384613


In [119]:

logit_gaussian3 = df_cifar.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_cifar.iloc[10].values.flatten().tolist()[1:]

def compute_metrics2(ap2a, k, l, ap2b, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6, value7 in zip(ap2a,ap2b):
        if value6<k or value6>l:
            FP +=1
        else:
            if value7<m or value7>n:
                FP +=1

    return (FP/(len(ap2a)))*100

#logitgaussian3
k=[44,64,76]
l=[405,405,405]

#attrgaussian3
m=[2200,2800,3100]
n=[8800,8800,8800]



print(compute_metrics2(
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0]))

print(compute_metrics2(
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1]))

print(compute_metrics2(
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2]))


3.076923076923077
10.384615384615385
20.76923076923077


In [120]:
#just 2a FPR

logit_gaussian3 = df_cifar.iloc[14].values.flatten().tolist()[1:]
#attr_gaussian3 = df_cifar.iloc[10].values.flatten().tolist()[1:]

def compute_metrics2(ap2a, k, l): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6 in (ap2a):
        if value6<k or value6>l:
            FP +=1

    return (FP/(len(ap2a)))*100

#logitgaussian3
k=[44,64,76]
l=[405,405,405]

#attrgaussian3
m=[2200,2800,3100]
n=[8800,8800,8800]



print(compute_metrics2(logit_gaussian3, k[0],l[0]))

print(compute_metrics2(logit_gaussian3, k[1],l[1]))

print(compute_metrics2(logit_gaussian3, k[2],l[2]))


2.307692307692308
5.5769230769230775
10.192307692307692


In [121]:
#just 2b FPR

#logit_gaussian3 = df_cifar.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_cifar.iloc[10].values.flatten().tolist()[1:]

def compute_metrics2(ap2a, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6 in (ap2a):
        if value6<m or value6>n:
            FP +=1

    return (FP/(len(ap2a)))*100

#logitgaussian3
k=[44,64,76]
l=[405,405,405]

#attrgaussian3
m=[2200,2800,3100]
n=[8800,8800,8800]



print(compute_metrics2(attr_gaussian3, m[0],n[0]))

print(compute_metrics2(attr_gaussian3, m[1],n[1]))

print(compute_metrics2(attr_gaussian3, m[2],n[2]))


0.7692307692307693
5.1923076923076925
11.346153846153847


# using all statistical detectors

In [122]:
#all statistical approaches
meanAbsDev = df_cifar.iloc[1].values.flatten().tolist()[1:]
medianAbsDev = df_cifar.iloc[0].values.flatten().tolist()[1:]
iqr = df_cifar.iloc[2].values.flatten().tolist()[1:]
coef_iqr = df_cifar.iloc[4].values.flatten().tolist()[1:]
coef_var = df_cifar.iloc[3].values.flatten().tolist()[1:]
squeezer_input = df_cifar.iloc[7].values.flatten().tolist()[1:]
logit_gaussian3 = df_cifar.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_cifar.iloc[10].values.flatten().tolist()[1:]

a=[1.3,1.85,2]
b=[7,7,6]
c=[0.9,1.08,1.23]
d=[5,4,4]
e=[0.461,0.491,0.502]
f=[0.8,0.8,0.8]
g=[0.42,0.42,0.42]
h=[1.27,1.19,1.146]


#iqr
o=[2,2.5,2.8]
p=[10,10,10]


def compute_metrics1(meanAb, a, b, medianAb, c,d, coefiqr, e, f, coefvar, g, h,iqr, o, p): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1, value2, value3, value4, value8 in zip(meanAb,medianAb,coefiqr,coefvar,iqr): 
        if value1<a or value1>b:
            FP += 1
        else:
            if value2<c or value2>d:
                FP +=1
            else: 
                if value3<e or value3>f:
                    FP +=1
                else:
                    if value4<g or value4>h:
                        FP +=1
                    else:
                        if value8<o or value8>p:
                            FP +=1

    
    
    return (FP/(len(meanAb)))*100


print(compute_metrics1(meanAbsDev, a[0], b[0],
                      medianAbsDev ,c[0], d[0],
                      coef_iqr , e[0], f[0], 
                      coef_var, g[0], h[0],
                    
                      iqr, o[0], p[0]))

print(compute_metrics1(meanAbsDev, a[1], b[1], 
                      medianAbsDev, c[1], d[1], 
                      coef_iqr, e[1], f[1],
                      coef_var, g[1], h[1], 
                      
                    
                      iqr, o[1], p[1]))

print(compute_metrics1(meanAbsDev, a[2], b[2],
                      medianAbsDev , c[2], d[2],
                      coef_iqr , e[2], f[2], 
                      coef_var,g[2], h[2],
                     
                     
                      iqr, o[2], p[2]))

3.076923076923077
15.0
26.153846153846157


In [123]:
#using mean abs dev

#all statistical approaches
meanAbsDev = df_cifar.iloc[1].values.flatten().tolist()[1:]
medianAbsDev = df_cifar.iloc[0].values.flatten().tolist()[1:]
iqr = df_cifar.iloc[2].values.flatten().tolist()[1:]
coef_iqr = df_cifar.iloc[4].values.flatten().tolist()[1:]
coef_var = df_cifar.iloc[3].values.flatten().tolist()[1:]
squeezer_input = df_cifar.iloc[7].values.flatten().tolist()[1:]
logit_gaussian3 = df_cifar.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_cifar.iloc[10].values.flatten().tolist()[1:]

a=[1.3,1.85,2]
b=[7,7,6]
c=[0.9,1.08,1.23]
d=[5,4,4]
e=[0.461,0.491,0.502]
f=[0.8,0.8,0.8]
g=[0.42,0.42,0.42]
h=[1.27,1.19,1.146]


#iqr
o=[2,2.5,2.8]
p=[10,10,10]


def compute_metrics1(meanAb, a, b): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1 in (meanAb): 
        if value1<a or value1>b:
            FP += 1
        
    return (FP/(len(meanAb)))*100


print(compute_metrics1(meanAbsDev, a[0], b[0]))

print(compute_metrics1(meanAbsDev, a[1], b[1]))

print(compute_metrics1(meanAbsDev, a[2], b[2]))

0.9615384615384616
5.1923076923076925
9.038461538461538


In [124]:
#median abs dev

meanAbsDev = df_cifar.iloc[1].values.flatten().tolist()[1:]
medianAbsDev = df_cifar.iloc[0].values.flatten().tolist()[1:]
iqr = df_cifar.iloc[2].values.flatten().tolist()[1:]
coef_iqr = df_cifar.iloc[4].values.flatten().tolist()[1:]
coef_var = df_cifar.iloc[3].values.flatten().tolist()[1:]
squeezer_input = df_cifar.iloc[7].values.flatten().tolist()[1:]
logit_gaussian3 = df_cifar.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_cifar.iloc[10].values.flatten().tolist()[1:]

a=[1.3,1.85,2]
b=[7,7,6]
c=[0.9,1.08,1.23]
d=[5,4,4]
e=[0.461,0.491,0.502]
f=[0.8,0.8,0.8]
g=[0.42,0.42,0.42]
h=[1.27,1.19,1.146]


#iqr
o=[2,2.5,2.8]
p=[10,10,10]


def compute_metrics1(medianAb, c,d): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value2 in (medianAb): 
        if value2<c or value2>d:
            FP += 1
    
    
    
    return (FP/(len(medianAb)))*100


print(compute_metrics1(
                      medianAbsDev ,c[0], d[0]))

print(compute_metrics1(
                      medianAbsDev, c[1], d[1]))

print(compute_metrics1(
                      medianAbsDev , c[2], d[2]))

0.576923076923077
4.807692307692308
10.576923076923077


In [125]:
#iqr  approaches

meanAbsDev = df_cifar.iloc[1].values.flatten().tolist()[1:]
medianAbsDev = df_cifar.iloc[0].values.flatten().tolist()[1:]
iqr = df_cifar.iloc[2].values.flatten().tolist()[1:]
coef_iqr = df_cifar.iloc[4].values.flatten().tolist()[1:]
coef_var = df_cifar.iloc[3].values.flatten().tolist()[1:]
squeezer_input = df_cifar.iloc[7].values.flatten().tolist()[1:]
logit_gaussian3 = df_cifar.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_cifar.iloc[10].values.flatten().tolist()[1:]

a=[1.3,1.85,2]
b=[7,7,6]
c=[0.9,1.08,1.23]
d=[5,4,4]
e=[0.461,0.491,0.502]
f=[0.8,0.8,0.8]
g=[0.42,0.42,0.42]
h=[1.27,1.19,1.146]


#iqr
o=[2,2.5,2.8]
p=[10,10,10]


def compute_metrics1(iqr, o, p): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value8 in (iqr): 
        if value8<o or value8>p:
            FP += 1
    
    return (FP/(len(iqr)))*100


print(compute_metrics1(
                      iqr, o[0], p[0]))

print(compute_metrics1(
                    
                      iqr, o[1], p[1]))

print(compute_metrics1(
                      iqr, o[2], p[2]))

0.576923076923077
3.4615384615384617
10.192307692307692


In [126]:
#coef iqr
meanAbsDev = df_cifar.iloc[1].values.flatten().tolist()[1:]
medianAbsDev = df_cifar.iloc[0].values.flatten().tolist()[1:]
iqr = df_cifar.iloc[2].values.flatten().tolist()[1:]
coef_iqr = df_cifar.iloc[4].values.flatten().tolist()[1:]
coef_var = df_cifar.iloc[3].values.flatten().tolist()[1:]


a=[1.3,1.85,2]
b=[7,7,6]
c=[0.9,1.08,1.23]
d=[5,4,4]
e=[0.461,0.491,0.502]
f=[0.8,0.8,0.8]
g=[0.42,0.42,0.42]
h=[1.27,1.19,1.146]


#iqr
o=[2,2.5,2.8]
p=[10,10,10]


def compute_metrics1(coefiqr, e, f): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value3 in (coefiqr): 
        if value3<e or value3>f:
            FP += 1
    
    return (FP/(len(coefiqr)))*100


print(compute_metrics1(
                      coef_iqr, e[0], f[0]))
                    

print(compute_metrics1(
                      coef_iqr, e[1], f[1]))

print(compute_metrics1(
                      coef_iqr , e[2], f[2]))

0.9615384615384616
5.1923076923076925
8.846153846153847


In [127]:
#coef var
meanAbsDev = df_cifar.iloc[1].values.flatten().tolist()[1:]
medianAbsDev = df_cifar.iloc[0].values.flatten().tolist()[1:]
iqr = df_cifar.iloc[2].values.flatten().tolist()[1:]
coef_iqr = df_cifar.iloc[4].values.flatten().tolist()[1:]
coef_var = df_cifar.iloc[3].values.flatten().tolist()[1:]
squeezer_input = df_cifar.iloc[7].values.flatten().tolist()[1:]
logit_gaussian3 = df_cifar.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_cifar.iloc[10].values.flatten().tolist()[1:]

a=[1.3,1.85,2]
b=[7,7,6]
c=[0.9,1.08,1.23]
d=[5,4,4]
e=[0.461,0.491,0.502]
f=[0.8,0.8,0.8]
g=[0.42,0.42,0.42]
h=[1.27,1.19,1.146]


#iqr
o=[2,2.5,2.8]
p=[10,10,10]


def compute_metrics1(coefvar, g, h): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value4 in (coefvar): 
        if value4<g or value4>h:
            FP += 1   
    
    return (FP/(len(coefvar)))*100


print(compute_metrics1(
                      coef_var, g[0], h[0]))

print(compute_metrics1(
                      coef_var, g[1], h[1]))

print(compute_metrics1(
                      coef_var,g[2], h[2]))

1.153846153846154
5.961538461538462
10.192307692307692


In [14]:
# what if i add one statistical appraoch? coef of variance. 

# so using only second approach. we have results already. lets check FPR

coef_var = df_cifar.iloc[3].values.flatten().tolist()[1:]
logit_gaussian3 = df_cifar.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_cifar.iloc[10].values.flatten().tolist()[1:]

def compute_metrics2(ap2a, k, l, ap2b, m, n, coefvar, g, h): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6, value7, value8 in zip(ap2a,ap2b,coefvar):
        if value6<k or value6>l:
            FP +=1
        else:
            if value7<m or value7>n:
                FP +=1
            else:
                if value8<g or value8>h:
                    FP+=1
                
    return (FP/(len(ap2a)))*100


#coef of var
g=[0.42,0.42,0.42]
h=[1.27,1.19,1.146]

#logitgaussian3
k=[44,64,76]
l=[405,405,405]

#attrgaussian3
m=[2200,2800,3100]
n=[8800,8800,8800]



print(compute_metrics2(
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0],
coef_var, g[0], h[0]))

print(compute_metrics2(
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1],coef_var, g[1], h[1]))

print(compute_metrics2(
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2], coef_var, g[2], h[2]))


4.038461538461538
14.807692307692308
27.884615384615387


# MNIST

In [86]:
feature_squeezer = df_mnist.iloc[16].values.flatten().tolist()[1:]
threshold = [0.54005, 0.540058, 0.05971]


def compute_metrics(ben, threshold): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in ben: 
        if value<threshold:
            TN += 1
        else: 
            FP += 1
    
    
    return (FP/(TN+FP))*100

In [87]:
for th in threshold:
    FPR = compute_metrics(feature_squeezer, th)
    print(FPR)

5.1923076923076925
5.1923076923076925
11.153846153846155


In [88]:
#using all approaches
meanAbsDev = df_mnist.iloc[1].values.flatten().tolist()[1:]
medianAbsDev = df_mnist.iloc[0].values.flatten().tolist()[1:]
iqr = df_mnist.iloc[2].values.flatten().tolist()[1:]
coef_iqr = df_mnist.iloc[4].values.flatten().tolist()[1:]
coef_var = df_mnist.iloc[3].values.flatten().tolist()[1:]
squeezer_input = df_mnist.iloc[7].values.flatten().tolist()[1:]
logit_gaussian3 = df_mnist.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_mnist.iloc[10].values.flatten().tolist()[1:]


def compute_metrics1(meanAb, a, b, medianAb, c,d, coefiqr, e, f, coefvar, g, h, ap1, i, j, ap2a, k, l, ap2b, m, n, iqr, o, p): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1, value2, value3, value4, value5, value6, value7, value8 in zip(meanAb,medianAb,coefiqr,coefvar,ap1,ap2a,ap2b, iqr): 
        if value1<a or value1>b:
            FP += 1
        else:
            if value2<c or value2>d:
                FP +=1
            else: 
                if value3<e or value3>f:
                    FP +=1
                else:
                    if value4<g or value4>h:
                        FP +=1
                    else: 
                        if value5<i or value5>j:
                            FP +=1
                        else: 
                            if value6<k or value6>l:
                                FP +=1
                            else:
                                if value7<m or value7>n:
                                    FP +=1
                                else:
                                    if value8<o or value8>p:
                                        FP +=1

    
    
    return (FP/(len(meanAb)))*100

In [89]:
a=[0.2,0.29,0.34]
b=[1.3,1.3,1.3]
c=[0.0,0.0,0.0]
d=[0.29,0.23,0.2]
e=[0.99,0.999,0.9999]
f=[1.1,1.1,1.0]
g=[0.34,0.34,0.34]
h=[0.73,0.69,0.67]

#app1
i=[16,25,35]
j=[370,370,370]

#app2a:logitgaussian3
k=[0.07,0.07,0.07]
l=[3.3,2.1,1.7]

#app2b:attrgaussian3
m=[0.1,0.1,0.1]
n=[250,150,120]

#iqr
o=[0.20,0.32,0.37]
p=[1.24,1.34,1.34]

In [90]:
print(compute_metrics1(meanAbsDev, a[0], b[0],
                      medianAbsDev ,c[0], d[0],
                      coef_iqr , e[0], f[0], 
                      coef_var, g[0], h[0],
                      squeezer_input,i[0],j[0], 
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0], 
                      iqr, o[0], p[0]))

print(compute_metrics1(meanAbsDev, a[1], b[1], 
                      medianAbsDev, c[1], d[1], 
                      coef_iqr, e[1], f[1],
                      coef_var, g[1], h[1], 
                      squeezer_input,i[1],j[1], 
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1], 
                      iqr, o[1], p[1]))

print(compute_metrics1(meanAbsDev, a[2], b[2],
                      medianAbsDev , c[2], d[2],
                      coef_iqr , e[2], f[2], 
                      coef_var,g[2], h[2],
                      squeezer_input,i[2],j[2], 
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2], 
                      iqr, o[2], p[2]))

93.46153846153847
99.8076923076923
100.0


In [93]:
#using only approach 1


squeezer_input = df_mnist.iloc[7].values.flatten().tolist()[1:]

def compute_metrics(ap1, i, j): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value5 in (ap1):
        if value5<i or value5>j:
            FP +=1
        

    return (FP/(len(ap1)))*100


#app1
i=[16,25,35]
j=[370,370,370]

#app2a:logitgaussian3
k=[0.07,0.07,0.07]
l=[3.3,2.1,1.7]

#app2b:attrgaussian3
m=[0.1,0.1,0.1]
n=[250,150,120]



print(compute_metrics(squeezer_input,i[0],j[0]))

print(compute_metrics(squeezer_input,i[1],j[1]))

print(compute_metrics(squeezer_input,i[2],j[2]))


1.7307692307692308
3.653846153846154
7.5


In [94]:
#just using approach 1 and approach 2
squeezer_input = df_mnist.iloc[7].values.flatten().tolist()[1:]
logit_gaussian3 = df_mnist.iloc[12].values.flatten().tolist()[1:]
attr_gaussian3 = df_mnist.iloc[8].values.flatten().tolist()[1:]


def compute_metrics2(ap1, i, j, ap2a, k, l, ap2b, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value5, value6, value7 in zip(ap1,ap2a,ap2b):
        if value5<i or value5>j:
            FP +=1
        else: 
            if value6<k or value6>l:
                FP +=1
            else:
                if value7<m or value7>n:
                    FP +=1

    return (FP/(len(ap1)))*100



print(compute_metrics2(squeezer_input,i[0],j[0], 
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0]))

print(compute_metrics2(squeezer_input,i[1],j[1], 
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1]))

print(compute_metrics2(squeezer_input,i[2],j[2], 
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2]))


5.5769230769230775
13.076923076923078
24.807692307692307


In [95]:
#just using approach 2

logit_gaussian3 = df_mnist.iloc[12].values.flatten().tolist()[1:]
attr_gaussian3 = df_mnist.iloc[8].values.flatten().tolist()[1:]


def compute_metrics2(ap2a, k, l, ap2b, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6, value7 in zip(ap2a,ap2b):
        if value6<k or value6>l:
            FP +=1
        else:
            if value7<m or value7>n:
                FP +=1

    return (FP/(len(ap2a)))*100


print(compute_metrics2( 
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0]))

print(compute_metrics2(
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1]))

print(compute_metrics2( 
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2]))



3.8461538461538463
10.0
18.461538461538463


In [96]:
#just using approach 2(a)

#just using approach 2

logit_gaussian3 = df_mnist.iloc[12].values.flatten().tolist()[1:]


def compute_metrics2(ap2a, k, l): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6 in (ap2a):
        if value6<k or value6>l:
            FP +=1
        
    return (FP/(len(ap2a)))*100


print(compute_metrics2( 
                      logit_gaussian3, k[0],l[0],
                     ))

print(compute_metrics2(
                      logit_gaussian3, k[1],l[1],
                    ))

print(compute_metrics2( 
                      logit_gaussian3, k[2],l[2],
                    ))



1.7307692307692308
6.538461538461539
11.73076923076923


In [98]:
#just using approach 2(b)

#just using approach 2

attr_gaussian3 = df_mnist.iloc[8].values.flatten().tolist()[1:]


def compute_metrics2( ap2b, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value7 in (ap2b):
        if value7<m or value7>n:
            FP +=1
    
    return (FP/(len(ap2b)))*100


print(compute_metrics2( 
                     
                      attr_gaussian3,m[0], n[0]))

print(compute_metrics2(
                     
                     attr_gaussian3,m[1], n[1]))

print(compute_metrics2( 
                     
                     attr_gaussian3,m[2], n[2]))



2.1153846153846154
4.615384615384616
9.230769230769232


In [83]:
#all statistical approaches
meanAbsDev = df_mnist.iloc[1].values.flatten().tolist()[1:]
medianAbsDev = df_mnist.iloc[0].values.flatten().tolist()[1:]
iqr = df_mnist.iloc[2].values.flatten().tolist()[1:]
coef_iqr = df_mnist.iloc[4].values.flatten().tolist()[1:]
coef_var = df_mnist.iloc[3].values.flatten().tolist()[1:]
squeezer_input = df_mnist.iloc[7].values.flatten().tolist()[1:]
logit_gaussian3 = df_mnist.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_mnist.iloc[10].values.flatten().tolist()[1:]


def compute_metrics1(meanAb, a, b, medianAb, c,d, coefiqr, e, f, coefvar, g, h,iqr, o, p): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1, value2, value3, value4, value8 in zip(meanAb,medianAb,coefiqr,coefvar,iqr): 
        if value1<a or value1>b:
            FP += 1
        else:
            if value2<c or value2>d:
                FP +=1
            else: 
                if value3<e or value3>f:
                    FP +=1
                else:
                    if value4<g or value4>h:
                        FP +=1
                    else:
                        if value8<o or value8>p:
                            FP +=1

    
    
    return (FP/(len(meanAb)))*100


print(compute_metrics1(meanAbsDev, a[0], b[0],
                      medianAbsDev ,c[0], d[0],
                      coef_iqr , e[0], f[0], 
                      coef_var, g[0], h[0],
                    
                      iqr, o[0], p[0]))

print(compute_metrics1(meanAbsDev, a[1], b[1], 
                      medianAbsDev, c[1], d[1], 
                      coef_iqr, e[1], f[1],
                      coef_var, g[1], h[1], 
                      
                    
                      iqr, o[1], p[1]))

print(compute_metrics1(meanAbsDev, a[2], b[2],
                      medianAbsDev , c[2], d[2],
                      coef_iqr , e[2], f[2], 
                      coef_var,g[2], h[2],
                     
                     
                      iqr, o[2], p[2]))

5.769230769230769
20.192307692307693
30.0


In [99]:
a=[0.2,0.29,0.34]
b=[1.3,1.3,1.3]
c=[0.0,0.0,0.0]
d=[0.29,0.23,0.2]
e=[0.99,0.999,0.9999]
f=[1.1,1.1,1.0]
g=[0.34,0.34,0.34]
h=[0.73,0.69,0.67]

#app1
i=[16,25,35]
j=[370,370,370]

#app2a:logitgaussian3
k=[0.07,0.07,0.07]
l=[3.3,2.1,1.7]

#app2b:attrgaussian3
m=[0.1,0.1,0.1]
n=[250,150,120]

#iqr
o=[0.20,0.32,0.37]
p=[1.24,1.34,1.34]

In [100]:
meanAbsDev = df_mnist.iloc[1].values.flatten().tolist()[1:]
def compute_metrics1(meanAb, a, b): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1 in (meanAb): 
        if value1<a or value1>b:
            FP += 1
        
    return (FP/(len(meanAb)))*100


print(compute_metrics1(meanAbsDev, a[0], b[0]))

print(compute_metrics1(meanAbsDev, a[1], b[1]))

print(compute_metrics1(meanAbsDev, a[2], b[2]))

1.7307692307692308
8.269230769230768
14.423076923076922


In [110]:
medianAbsDev = df_mnist.iloc[0].values.flatten().tolist()[1:]
def compute_metrics1(medianAb, c,d): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value2 in (medianAb): 
        if value2<c or value2>d:
            FP += 1
    
    
    
    return (FP/(len(medianAb)))*100


print(compute_metrics1(
                      medianAbsDev ,c[0], d[0]))

print(compute_metrics1(
                      medianAbsDev, c[1], d[1]))

print(compute_metrics1(
                      medianAbsDev , c[2], d[2]))

1.153846153846154
5.769230769230769
10.0


In [109]:
iqr = df_mnist.iloc[2].values.flatten().tolist()[1:]
def compute_metrics1(iqr, o, p): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value8 in (iqr): 
        if value8<o or value8>p:
            FP += 1
    
    return (FP/(len(iqr)))*100


print(compute_metrics1(
                      iqr, o[0], p[0]))

print(compute_metrics1(
                    
                      iqr, o[1], p[1]))

print(compute_metrics1(
                      iqr, o[2], p[2]))

1.3461538461538463
9.423076923076923
13.846153846153847


In [107]:
coef_iqr = df_mnist.iloc[4].values.flatten().tolist()[1:]
def compute_metrics1(coefiqr, e, f): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value3 in (coefiqr): 
        if value3<e or value3>f:
            FP += 1
    
    return (FP/(len(coefiqr)))*100


print(compute_metrics1(
                      coef_iqr, e[0], f[0]))
                    

print(compute_metrics1(
                      coef_iqr, e[1], f[1]))

print(compute_metrics1(
                      coef_iqr , e[2], f[2]))

3.2692307692307696
3.653846153846154
3.653846153846154


In [108]:
coef_var = df_mnist.iloc[3].values.flatten().tolist()[1:]
def compute_metrics1(coefvar, g, h): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value4 in (coefvar): 
        if value4<g or value4>h:
            FP += 1   
    
    return (FP/(len(coefvar)))*100


print(compute_metrics1(
                      coef_var, g[0], h[0]))

print(compute_metrics1(
                      coef_var, g[1], h[1]))

print(compute_metrics1(
                      coef_var,g[2], h[2]))

0.7692307692307693
3.8461538461538463
6.923076923076923


In [84]:
#all statistical approaches except iqr
meanAbsDev = df_mnist.iloc[1].values.flatten().tolist()[1:]
medianAbsDev = df_mnist.iloc[0].values.flatten().tolist()[1:]
iqr = df_mnist.iloc[2].values.flatten().tolist()[1:]
coef_iqr = df_mnist.iloc[4].values.flatten().tolist()[1:]
coef_var = df_mnist.iloc[3].values.flatten().tolist()[1:]
squeezer_input = df_mnist.iloc[7].values.flatten().tolist()[1:]
logit_gaussian3 = df_mnist.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_mnist.iloc[10].values.flatten().tolist()[1:]


def compute_metrics1(meanAb, a, b, medianAb, c,d, coefiqr, e, f, coefvar, g, h): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1, value2, value3, value4 in zip(meanAb,medianAb,coefiqr,coefvar): 
        if value1<a or value1>b:
            FP += 1
        else:
            if value2<c or value2>d:
                FP +=1
            else: 
                if value3<e or value3>f:
                    FP +=1
                else:
                    if value4<g or value4>h:
                        FP +=1

    
    
    return (FP/(len(meanAb)))*100


print(compute_metrics1(meanAbsDev, a[0], b[0],
                      medianAbsDev ,c[0], d[0],
                      coef_iqr , e[0], f[0], 
                      coef_var, g[0], h[0],
                    ))

print(compute_metrics1(meanAbsDev, a[1], b[1], 
                      medianAbsDev, c[1], d[1], 
                      coef_iqr, e[1], f[1],
                      coef_var, g[1], h[1], 
                      ))

print(compute_metrics1(meanAbsDev, a[2], b[2],
                      medianAbsDev , c[2], d[2],
                      coef_iqr , e[2], f[2], 
                      coef_var,g[2], h[2],
                     ))

5.5769230769230775
17.307692307692307
27.307692307692307


In [85]:
#all statistical approaches except iqr and mean abs dev

medianAbsDev = df_mnist.iloc[0].values.flatten().tolist()[1:]
iqr = df_mnist.iloc[2].values.flatten().tolist()[1:]
coef_iqr = df_mnist.iloc[4].values.flatten().tolist()[1:]
coef_var = df_mnist.iloc[3].values.flatten().tolist()[1:]
squeezer_input = df_mnist.iloc[7].values.flatten().tolist()[1:]
logit_gaussian3 = df_mnist.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_mnist.iloc[10].values.flatten().tolist()[1:]


def compute_metrics1(medianAb, c,d, coefiqr, e, f, coefvar, g, h): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value2, value3, value4 in zip(medianAb,coefiqr,coefvar): 
        if value2<c or value2>d:
            FP +=1
        else: 
            if value3<e or value3>f:
                FP +=1
            else:
                if value4<g or value4>h:
                    FP +=1

    

    return (FP/(len(medianAb)))*100


print(compute_metrics1(
                      medianAbsDev ,c[0], d[0],
                      coef_iqr , e[0], f[0], 
                      coef_var, g[0], h[0],
                    ))

print(compute_metrics1( 
                      medianAbsDev, c[1], d[1], 
                      coef_iqr, e[1], f[1],
                      coef_var, g[1], h[1], 
                      ))

print(compute_metrics1(
                      medianAbsDev , c[2], d[2],
                      coef_iqr , e[2], f[2], 
                      coef_var,g[2], h[2],
                     ))

4.038461538461538
9.615384615384617
14.615384615384617


In [36]:
# statistical approaches using coef of var and coef of iqr only 

coef_iqr = df_mnist.iloc[4].values.flatten().tolist()[1:]
coef_var = df_mnist.iloc[3].values.flatten().tolist()[1:]



def compute_metrics1(coefiqr, e, f, coefvar, g, h): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for  value3, value4 in zip(coefiqr,coefvar): 
        if value3<e or value3>f:
            FP +=1
        else:
            if value4<g or value4>h:
                FP +=1

    

    return (FP/(len(coefiqr)))*100


print(compute_metrics1(
                     
                      coef_iqr , e[0], f[0], 
                      coef_var, g[0], h[0],
                    ))

print(compute_metrics1( 
                     
                      coef_iqr, e[1], f[1],
                      coef_var, g[1], h[1], 
                      ))

print(compute_metrics1(
                     
                      coef_iqr , e[2], f[2], 
                      coef_var,g[2], h[2],
                     ))

3.653846153846154
6.153846153846154
9.038461538461538


# imagenet

In [128]:
df_imagenet

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,500,501,502,503,504,505,506,507,508,509
0,Benign Median Absolute Dev,0.011741,0.008387,0.020484,0.021316,0.008825,0.017424,0.021685,0.021631,0.016082,...,0.012902,0.033298,0.015329,0.019026,0.012905,0.019164,0.021244,0.018071,0.020315,0.025899
1,Benign Mean Absolute Dev,0.025300,0.014168,0.032768,0.041157,0.022436,0.030149,0.036664,0.040445,0.028866,...,0.022228,0.058719,0.028696,0.029467,0.020665,0.030322,0.035129,0.036164,0.033193,0.041576
2,Benign IQR,0.028198,0.019277,0.046132,0.052185,0.021889,0.040852,0.049536,0.050808,0.037256,...,0.030106,0.077184,0.037065,0.041414,0.028777,0.042910,0.048298,0.043622,0.045557,0.059069
3,Coefficient of Variance,0.744189,1.045666,1.071272,0.895818,0.670811,0.977936,1.021734,0.900896,0.944452,...,0.977874,0.961550,0.887167,1.139028,1.049775,1.091366,0.927226,0.875777,1.092827,1.086345
4,Coef of IQR,0.572312,0.543271,0.554579,0.591542,0.568094,0.587290,0.546192,0.562787,0.566464,...,0.594566,0.586153,0.608877,0.486604,0.538749,0.539617,0.588851,0.579019,0.507159,0.547000
5,Input squeeze top 100,2763.148926,2081.828369,1023.216736,3165.566895,2523.543457,1926.836670,2048.480225,3013.867676,2095.982178,...,2124.283447,2703.627441,2947.141113,5345.160156,2508.094971,1528.462280,2568.843750,2811.551758,4963.367188,1407.468872
6,Input squeeze top 150,3862.401367,2548.094482,1529.362061,4507.639648,3142.960449,2056.201660,2796.873047,4553.828125,2425.316895,...,2457.674805,3022.847656,3282.999756,6066.457031,3486.907471,2046.148804,3031.932129,3942.962402,5536.892578,1664.476562
7,Input squeeze top 200,5170.620605,4282.488770,3457.750000,6209.315430,4115.356445,2683.388672,3700.596680,5285.743652,4659.582031,...,2505.857178,4154.269043,4783.180664,6161.008789,4379.708984,2318.330078,3826.410400,6047.323242,7253.958008,2350.700195
8,Gaussian1 attribution,2058.083496,1512.591797,4591.742188,3610.160645,1759.417236,2427.506348,3457.222412,2805.120605,2660.168701,...,1926.328979,4939.623047,2015.094482,3138.607666,2141.564697,3485.207031,2988.595947,3139.858398,2598.096191,3340.587891
9,Gaussian2 attribution,1987.025879,1437.073364,4453.267578,2912.220459,1437.591309,2207.568115,2832.688721,2717.238037,2273.826904,...,1736.976807,4110.721680,1939.618896,2230.654053,2068.590820,2731.733887,2668.289795,2404.806885,2413.304688,4177.527344


In [129]:
feature_squeezer = df_imagenet.iloc[16].values.flatten().tolist()[1:]
threshold = [1.36418, 1.3909, 1.299158]

def compute_metrics(ben, threshold): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in ben: 
        if value<threshold:
            TN += 1
        else: 
            FP += 1
    
    
    return (FP/(TN+FP))*100

In [130]:
for th in threshold:
    FPR = compute_metrics(feature_squeezer, th)
    print(FPR)

12.941176470588237
11.76470588235294
15.294117647058824


In [131]:
a=[0.015,0.019,0.022]
b=[0.11,0.11,0.11]

In [132]:
meanAbsDev = df_imagenet.iloc[1].values.flatten().tolist()[1:]
medianAbsDev = df_imagenet.iloc[0].values.flatten().tolist()[1:]
iqr = df_imagenet.iloc[2].values.flatten().tolist()[1:]
coef_iqr = df_imagenet.iloc[4].values.flatten().tolist()[1:]
coef_var = df_imagenet.iloc[3].values.flatten().tolist()[1:]

squeezer_input = df_imagenet.iloc[6].values.flatten().tolist()[1:]

logit_gaussian3 = df_imagenet.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_imagenet.iloc[10].values.flatten().tolist()[1:]

In [133]:
def compute_metrics(meanAb, a, b, medianAb, c,d, coefiqr, e, f, coefvar, g, h, ap1, i, j, ap2a, k, l, ap2b, m, n, iqr, o, p): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1, value2, value3, value4, value5, value6, value7, value8 in zip(meanAb,medianAb,coefiqr,coefvar,ap1,ap2a,ap2b, iqr): 
        if value1<a or value1>b:
            FP += 1
        else:
            if value2<c or value2>d:
                FP +=1
            else: 
                if value3<e or value3>f:
                    FP +=1
                else:
                    if value4<g or value4>h:
                        FP +=1
                    else: 
                        if value5<i or value5>j:
                            FP +=1
                        else: 
                            if value6<k or value6>l:
                                FP +=1
                            else:
                                if value7<m or value7>n:
                                    FP +=1
                                else:
                                    if value8<o or value8>p:
                                        FP +=1

    
    
    return (FP/(len(meanAb)))*100

In [134]:
a=[0.015,0.019,0.022]
b=[0.11,0.11,0.11]
c=[0.008,0.01,0.0115]
d=[0.055,0.055,0.055]
e=[0.49,0.516,0.5257]
f=[0.74,0.74,0.74]
g=[0.38,0.38,0.38]
h=[1.14,1.09,1.07]


i=[1300,1800,2000]
j=[5700,5700,5700]

k=[1810,2510,2810]
l=[6700,6700,6700]

m=[1400,1750,1900]
n=[6600,6600,6600]


#iqr
o=[0.019,0.024,0.027]
p=[0.13,0.13,0.13]


In [135]:
print(compute_metrics(meanAbsDev, a[0], b[0],
                      medianAbsDev ,c[0], d[0],
                      coef_iqr , e[0], f[0], 
                      coef_var, g[0], h[0],
                      squeezer_input,i[0],j[0], 
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0], 
                      iqr, o[0], p[0]))

print(compute_metrics(meanAbsDev, a[1], b[1], 
                      medianAbsDev, c[1], d[1], 
                      coef_iqr, e[1], f[1],
                      coef_var, g[1], h[1], 
                      squeezer_input,i[1],j[1], 
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1], 
                      iqr, o[1], p[1]))

print(compute_metrics(meanAbsDev, a[2], b[2],
                      medianAbsDev , c[2], d[2],
                      coef_iqr , e[2], f[2], 
                      coef_var,g[2], h[2],
                      squeezer_input,i[2],j[2], 
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2], 
                      iqr, o[2], p[2]))

9.019607843137255
28.627450980392155
42.745098039215684


In [136]:
#using statistical approach only 

def compute_metrics(meanAb, a, b, medianAb, c,d, coefiqr, e, f, coefvar, g, h, iqr, o, p): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1, value2, value3, value4, value8 in zip(meanAb,medianAb,coefiqr,coefvar, iqr): 
        if value1<a or value1>b:
            FP += 1
        else:
            if value2<c or value2>d:
                FP +=1
            else: 
                if value3<e or value3>f:
                    FP +=1
                else:
                    if value4<g or value4>h:
                        FP +=1
                    else:
                        if value8<o or value8>p:
                            FP +=1

    
    
    return (FP/(len(meanAb)))*100


print(compute_metrics(meanAbsDev, a[0], b[0],
                      medianAbsDev ,c[0], d[0],
                      coef_iqr , e[0], f[0], 
                      coef_var, g[0], h[0],
                      
                      iqr, o[0], p[0]))

print(compute_metrics(meanAbsDev, a[1], b[1], 
                      medianAbsDev, c[1], d[1], 
                      coef_iqr, e[1], f[1],
                      coef_var, g[1], h[1], 
                     
                      iqr, o[1], p[1]))

print(compute_metrics(meanAbsDev, a[2], b[2],
                      medianAbsDev , c[2], d[2],
                      coef_iqr , e[2], f[2], 
                      coef_var,g[2], h[2],
                     
                      iqr, o[2], p[2]))


2.941176470588235
12.549019607843137
23.52941176470588


In [145]:
meanAbsDev = df_imagenet.iloc[1].values.flatten().tolist()[1:]
def compute_metrics1(meanAb, a, b): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1 in (meanAb): 
        if value1<a or value1>b:
            FP += 1
        
    return (FP/(len(meanAb)))*100


print(compute_metrics1(meanAbsDev, a[0], b[0]))

print(compute_metrics1(meanAbsDev, a[1], b[1]))

print(compute_metrics1(meanAbsDev, a[2], b[2]))

0.7843137254901961
4.117647058823529
9.607843137254903


In [146]:
medianAbsDev = df_imagenet.iloc[0].values.flatten().tolist()[1:]
def compute_metrics1(medianAb, c,d): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value2 in (medianAb): 
        if value2<c or value2>d:
            FP += 1
    
    
    
    return (FP/(len(medianAb)))*100


print(compute_metrics1(
                      medianAbsDev ,c[0], d[0]))

print(compute_metrics1(
                      medianAbsDev, c[1], d[1]))

print(compute_metrics1(
                      medianAbsDev , c[2], d[2]))

0.9803921568627451
4.705882352941177
9.215686274509805


In [147]:
iqr = df_imagenet.iloc[2].values.flatten().tolist()[1:]
def compute_metrics1(iqr, o, p): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value8 in (iqr): 
        if value8<o or value8>p:
            FP += 1
    
    return (FP/(len(iqr)))*100


print(compute_metrics1(
                      iqr, o[0], p[0]))

print(compute_metrics1(
                    
                      iqr, o[1], p[1]))

print(compute_metrics1(
                      iqr, o[2], p[2]))

1.1764705882352942
5.098039215686274
8.627450980392156


In [148]:
coef_iqr = df_imagenet.iloc[4].values.flatten().tolist()[1:]
def compute_metrics1(coefiqr, e, f): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value3 in (coefiqr): 
        if value3<e or value3>f:
            FP += 1
    
    return (FP/(len(coefiqr)))*100


print(compute_metrics1(
                      coef_iqr, e[0], f[0]))
                    

print(compute_metrics1(
                      coef_iqr, e[1], f[1]))

print(compute_metrics1(
                      coef_iqr , e[2], f[2]))

0.7843137254901961
4.313725490196078
8.03921568627451


In [149]:
coef_var = df_imagenet.iloc[3].values.flatten().tolist()[1:]
def compute_metrics1(coefvar, g, h): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value4 in (coefvar): 
        if value4<g or value4>h:
            FP += 1   
    
    return (FP/(len(coefvar)))*100


print(compute_metrics1(
                      coef_var, g[0], h[0]))

print(compute_metrics1(
                      coef_var, g[1], h[1]))

print(compute_metrics1(
                      coef_var,g[2], h[2]))

1.3725490196078431
5.294117647058823
8.823529411764707


In [137]:
#using approach 2 and statistical
def compute_metrics(meanAb, a, b, medianAb, c,d, coefiqr, e, f, coefvar, g, h, ap2a, k, l, ap2b, m, n, iqr, o, p): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1, value2, value3, value4, value6, value7, value8 in zip(meanAb,medianAb,coefiqr,coefvar,ap2a,ap2b, iqr): 
        if value1<a or value1>b:
            FP += 1
        else:
            if value2<c or value2>d:
                FP +=1
            else: 
                if value3<e or value3>f:
                    FP +=1
                else:
                    if value4<g or value4>h:
                        FP +=1
                    else:
                        if value6<k or value6>l:
                            FP +=1
                        else:
                            if value7<m or value7>n:
                                FP +=1
                            else:
                                if value8<o or value8>p:
                                    FP +=1

    
    
    return (FP/(len(meanAb)))*100

print(compute_metrics(meanAbsDev, a[0], b[0],
                      medianAbsDev ,c[0], d[0],
                      coef_iqr , e[0], f[0], 
                      coef_var, g[0], h[0],
                     
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0], 
                      iqr, o[0], p[0]))

print(compute_metrics(meanAbsDev, a[1], b[1], 
                      medianAbsDev, c[1], d[1], 
                      coef_iqr, e[1], f[1],
                      coef_var, g[1], h[1], 
                     
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1], 
                      iqr, o[1], p[1]))

print(compute_metrics(meanAbsDev, a[2], b[2],
                      medianAbsDev , c[2], d[2],
                      coef_iqr , e[2], f[2], 
                      coef_var,g[2], h[2],
                      
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2], 
                      iqr, o[2], p[2]))

7.8431372549019605
26.078431372549023
38.82352941176471


In [138]:
#using approach 2 and coef of var and coef of iqr
def compute_metrics( coefiqr, e, f, coefvar, g, h, ap2a, k, l, ap2b, m, n): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value3, value4, value6, value7 in zip(coefiqr,coefvar,ap2a,ap2b): 
        if value3<e or value3>f:
            FP +=1
        else:
            if value4<g or value4>h:
                FP +=1
            else:
                if value6<k or value6>l:
                    FP +=1
                else:
                    if value7<m or value7>n:
                        FP +=1
              

    
    
    return (FP/(len(coefiqr)))*100

print(compute_metrics(
                      coef_iqr , e[0], f[0], 
                      coef_var, g[0], h[0],
                     
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0], 
                      ))

print(compute_metrics(
                      coef_iqr, e[1], f[1],
                      coef_var, g[1], h[1], 
                     
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1], 
                    ))

print(compute_metrics(
                      coef_iqr , e[2], f[2], 
                      coef_var,g[2], h[2],
                      
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2], 
                      ))

7.8431372549019605
25.68627450980392
37.84313725490196


In [139]:
#using approach 2 and coef of var
def compute_metrics( coefvar, g, h, ap2a, k, l, ap2b, m, n): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value4, value6, value7 in zip(coefvar,ap2a,ap2b): 
        if value4<g or value4>h:
            FP +=1
        else:
            if value6<k or value6>l:
                FP +=1
            else:
                if value7<m or value7>n:
                    FP +=1
              

    
    
    return (FP/(len(coefvar)))*100

print(compute_metrics(
                      
                      coef_var, g[0], h[0],
                     
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0], 
                      ))

print(compute_metrics(
                     
                      coef_var, g[1], h[1], 
                     
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1], 
                    ))

print(compute_metrics(
                      
                      coef_var,g[2], h[2],
                      
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2], 
                      ))

7.450980392156863
24.313725490196077
35.68627450980392


In [283]:
#using approach 1 and 2 only
squeezer_input = df_imagenet.iloc[6].values.flatten().tolist()[1:]
logit_gaussian3 = df_imagenet.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_imagenet.iloc[10].values.flatten().tolist()[1:]

def compute_metrics2(ap1, i, j, ap2a, k, l, ap2b, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value5, value6, value7 in zip(ap1,ap2a,ap2b):
        if value5<i or value5>j:
            FP +=1
        else: 
            if value6<k or value6>l:
                FP +=1
            else:
                if value7<m or value7>n:
                    FP +=1

    return (FP/(len(ap1)))*100


print(compute_metrics2(squeezer_input,i[0],j[0], 
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0]))

print(compute_metrics2(squeezer_input,i[1],j[1], 
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1]))

print(compute_metrics2(squeezer_input,i[2],j[2], 
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2]))


7.647058823529412
23.72549019607843
34.509803921568626


In [144]:
#using approach 1  only
squeezer_input = df_imagenet.iloc[6].values.flatten().tolist()[1:]


def compute_metrics2(ap1, i, j): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value5 in (ap1):
        if value5<i or value5>j:
            FP +=1

    return (FP/(len(ap1)))*100


print(compute_metrics2(squeezer_input,i[0],j[0]
                      ))

print(compute_metrics2(squeezer_input,i[1],j[1]))

print(compute_metrics2(squeezer_input,i[2],j[2]))


2.156862745098039
5.686274509803922
12.352941176470589


In [140]:
#using approach 2 only
logit_gaussian3 = df_imagenet.iloc[14].values.flatten().tolist()[1:]
attr_gaussian3 = df_imagenet.iloc[10].values.flatten().tolist()[1:]

def compute_metrics2(ap2a, k, l, ap2b, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6, value7 in zip(ap2a,ap2b):
        if value6<k or value6>l:
            FP +=1
        else:
            if value7<m or value7>n:
                FP +=1

    return (FP/(len(ap2a)))*100


print(compute_metrics2( 
                      logit_gaussian3, k[0],l[0],
                      attr_gaussian3,m[0], n[0]))

print(compute_metrics2(
                      logit_gaussian3, k[1],l[1],
                     attr_gaussian3,m[1], n[1]))

print(compute_metrics2(
                      logit_gaussian3, k[2],l[2],
                     attr_gaussian3,m[2], n[2]))


6.2745098039215685
20.784313725490197
29.607843137254903


In [141]:
#using approach 2a only
logit_gaussian3 = df_imagenet.iloc[14].values.flatten().tolist()[1:]

def compute_metrics2(ap2a, k, l): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6 in (ap2a):
        if value6<k or value6>l:
            FP +=1
       
    return (FP/(len(ap2a)))*100


print(compute_metrics2( 
                      logit_gaussian3, k[0],l[0],
                      ))

print(compute_metrics2(
                      logit_gaussian3, k[1],l[1]
   ))

print(compute_metrics2(
                      logit_gaussian3, k[2],l[2],
                   ))


1.5686274509803921
6.470588235294119
10.392156862745098


In [143]:
#using approach 2b only
attr_gaussian3 = df_imagenet.iloc[10].values.flatten().tolist()[1:]

def compute_metrics2( ap2b, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value7 in (ap2b):
        if value7<m or value7>n:
            FP +=1
        
    return (FP/(len(ap2b)))*100


print(compute_metrics2( 
                     
                      attr_gaussian3,m[0], n[0]))

print(compute_metrics2(
                     
                     attr_gaussian3,m[1], n[1]))

print(compute_metrics2(
                     
                     attr_gaussian3,m[2], n[2]))


4.705882352941177
15.88235294117647
23.52941176470588
